In [23]:
import cv2
import numpy as np

In [24]:
mtx1 = np.load('ugreen_left_parameters/mtx.npy')
mtx2 = np.load('ugreen_right_parameters/mtx.npy')
dist1 = np.load('ugreen_left_parameters/dist.npy')
dist2 = np.load('ugreen_left_parameters/dist.npy')

In [25]:
rvecs = np.array([0, 0, 0])  # Keep as float32
tvecs = np.array([8.2, 0, 0])  # Keep as float32

# Define image size as a tuple of integers
imageSize = (640, 480)  # Image width and height as a tuple of integers

In [26]:
R1, R2, P1, P2, Q, validPixROI1, validPixROI2 = cv2.stereoRectify(
    mtx1, dist1, mtx2, dist2, imageSize, rvecs, tvecs
)
map1x, map1y = cv2.initUndistortRectifyMap(mtx1, dist1, R1, P1, imageSize, cv2.CV_32FC1)
map2x, map2y = cv2.initUndistortRectifyMap(mtx2, dist2, R2, P2, imageSize, cv2.CV_32FC1)

# Remap the images to rectify them

In [27]:
left = cv2.VideoCapture(0, cv2.CAP_DSHOW)
right = cv2.VideoCapture(1, cv2.CAP_DSHOW)


while True:
    
    ret1, frame1 = left.read()
    ret2, frame2 = right.read()

    cv2.imshow('Left', frame1)
    cv2.imshow('Right', frame2)

    rectified_left = cv2.remap(frame1, map1x, map1y, cv2.INTER_LINEAR)
    rectified_right = cv2.remap(frame2, map2x, map2y, cv2.INTER_LINEAR)

    undistorted_img_left = cv2.undistort(frame1, mtx1, dist1)
    undistorted_img_right = cv2.undistort(frame2, mtx2, dist2)

    # cv2.imshow('Undistorted_left', undistorted_img_left)
    # cv2.imshow('Undistorted_right', undistorted_img_right)
    window_size = 10

    # Disparity calculation with StereoSGBM
    # stereo = cv2.StereoSGBM_create(
    #     minDisparity=0,
    #     numDisparities=96,  # Should be a multiple of 16
    #     blockSize=5,
    #     P1=8 * 3 * window_size ** 2,  # Regularization term (adjust as needed)
    #     P2=32 * 3 * window_size ** 2,  # Smoothness term (adjust as needed)
    #     disp12MaxDiff=1,
    #     uniquenessRatio=5,
    #     speckleWindowSize=100,
    #     speckleRange=32
    # )

    stereo = cv2.StereoSGBM_create(
        minDisparity=0,
        numDisparities=80,  # Should be a multiple of 16
        blockSize=5,
        P1=8 * 3 * window_size ** 2,  # Regularization term (adjust as needed)
        P2=32 * 3 * window_size ** 2,  # Smoothness term (adjust as needed)
        disp12MaxDiff=1,
        uniquenessRatio=10,
        speckleWindowSize=100,
        speckleRange=32
    )

    # Compute the disparity map
    disparity_map = stereo.compute(rectified_left, rectified_right)

    # Reproject to 3D
    depth_map = cv2.reprojectImageTo3D(disparity_map, Q)

    # Display the disparity map
    cv2.imshow("rect_left", rectified_left)
    cv2.imshow("rect_right", rectified_right)
    cv2.imshow("Disparity Map", disparity_map / disparity_map.max())  # Normalize for better visualization
    # cv2.imshow('depth_map', depth_map)
    if cv2.waitKey(1) &  0xFF == 27:
        break

left.release()
right.release()

cv2.destroyAllWindows()
